In [0]:
spark.sql("DROP TABLE IF EXISTS mc.amadeus2.data")
df = spark.read.parquet("/Volumes/mc/amadeus2/data/part-00000-7118aad9-0739-4860-912f-1a58c85d689c.c000.snappy.parquet")
df.write.saveAsTable("mc.amadeus2.data")
display(spark.table("mc.amadeus2.data"))

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE mc.amadeus2.data_full_1 AS
SELECT
  d.*,
  i.city AS trip_origin_city_full,
  i.country AS trip_origin_country_full
FROM mc.amadeus2.data_iata d
LEFT JOIN mc.amadeus2.iata i
  ON d.trip_origin_city = i.iata
""")
display(spark.table("mc.amadeus2.data_full_1"))

In [0]:
spark.sql("ALTER TABLE mc.amadeus2.data_full RENAME TO mc.amadeus2.data_full_1")

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE mc.amadeus2.data_full AS
SELECT
  d.*,
  i.city AS trip_destination_city_full,
  i.country AS trip_destination_country_full
FROM mc.amadeus2.data_full_1 d
LEFT JOIN mc.amadeus2.iata i
  ON d.trip_origin_city = i.iata
""")
display(spark.table("mc.amadeus2.data_full"))

In [0]:
spark.sql("ALTER TABLE mc.amadeus2.data RENAME TO mc.amadeus2.data_iata")

In [0]:
spark.sql("ALTER TABLE mc.amadeus2.data_jan2026 RENAME TO mc.amadeus2.data_jan26")

In [0]:
spark.sql("DROP TABLE IF EXISTS mc.amadeus2.iata")

In [0]:
spark.sql("DROP TABLE IF EXISTS mc.amadeus2.data_full")

In [0]:
df_iata = spark.read.format("csv").option("header", True).load("/Volumes/mc/amadeus2/data/iata.csv")
df_iata.write.mode("overwrite").saveAsTable("mc.amadeus2.iata")
display(spark.table("mc.amadeus2.iata"))

In [0]:
display(spark.sql("SELECT * FROM mc.amadeus2.iata WHERE country = 'Ethiopia'"))

In [0]:
display(spark.sql("SELECT DISTINCT flight_leg_departure_date FROM mc.amadeus2.data"))

In [0]:
display(spark.sql("SELECT DISTINCT flight_leg_departure_date FROM mc.amadeus2.data_jan2026"))

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, "mc.amadeus2.data")
deltaTable.update(
    set = { "flight_leg_departure_date": "'2026-01-01'" }
)

In [0]:
date_range_df = spark.sql("""
SELECT explode(sequence(to_date('2026-01-01'), to_date('2026-01-31'), interval 1 day)) AS flight_leg_departure_date
""")

df_data = spark.table("mc.amadeus2.data").drop("flight_leg_departure_date")

df_expanded = df_data.crossJoin(date_range_df) \
    .withColumn("flight_leg_departure_date", date_range_df["flight_leg_departure_date"])

df_expanded.write.mode("overwrite").saveAsTable("mc.amadeus2.data_jan2026")

display(spark.table("mc.amadeus2.data_jan2026"))

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE mc.amadeus2.data AS
SELECT * FROM mc.amadeus2.data_full
""")

In [0]:
%sql
SELECT flight_leg_origin_country, COUNT(*) AS count FROM mc.amadeus2.data GROUP BY flight_leg_origin_country ORDER BY flight_leg_origin_country

In [0]:
display(spark.sql("""
SELECT trip_origin_city,
       trip_origin_country,
       trip_destination_city,
       trip_destination_country,
       SUM(flight_leg_total_seats) AS total_seats
FROM mc.amadeus2.data
GROUP BY trip_origin_city,
         trip_origin_country,
         trip_destination_city,
         trip_destination_country
"""))

In [0]:
display(spark.sql("""
SELECT trip_origin_city,
       trip_origin_city_full,
       trip_origin_country,
       trip_origin_country_full,
       trip_destination_city,
       trip_destination_city_full,
       trip_destination_country,
       trip_destination_country_full,
       trip_destination_city,
       trip_destination_country,
SUM(flight_leg_total_seats) AS total_seats
FROM mc.amadeus2.data_full
GROUP BY trip_origin_city,
         trip_origin_city_full,
         trip_origin_country,
         trip_origin_country_full,
         trip_destination_city,
         trip_destination_city_full,
         trip_destination_country,
         trip_destination_country_full,
         trip_destination_city,
         trip_destination_country
"""))

In [0]:
display(spark.sql("""
SELECT trip_origin_city,
       trip_origin_city_full,
       trip_origin_country,
       trip_origin_country_full,
       trip_destination_city,
       trip_destination_city_full,
       trip_destination_country,
       trip_destination_country_full,
       trip_destination_city,
       trip_destination_country,
SUM(flight_leg_total_seats) AS total_seats
FROM mc.amadeus2.data_full
WHERE trip_origin_country = 'FR' AND trip_destination_country = 'ES'
GROUP BY trip_origin_city,
         trip_origin_city_full,
         trip_origin_country,
         trip_origin_country_full,
         trip_destination_city,
         trip_destination_city_full,
         trip_destination_country,
         trip_destination_country_full,
         trip_destination_city,
         trip_destination_country
"""))

In [0]:
spark.table("mc.amadeus2.data_full")

In [0]:
display(spark.table("mc.amadeus2.data_full").selectExpr("count(*) as row_count"))

In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
import random

# Read the table and filter for the target date
df = spark.table("mc.amadeus2.data_jan26")
df_target_date = df.filter(F.col("flight_leg_departure_date") == "2026-01-29")

# Add a random column to select 10% of rows
df_with_random = df_target_date.withColumn("random_val", F.rand(seed=42))

# Select 10% of rows (random_val < 0.1)
df_selected = df_with_random.filter(F.col("random_val") < 0.1)

# Add another random column to decide between high/low outlier (50/50 split)
df_selected = df_selected.withColumn("outlier_type", F.rand(seed=123))

# Apply multipliers: if outlier_type < 0.5, use high multiplier (2.0-3.0), else low (0.3-0.5)
df_selected = df_selected.withColumn(
    "multiplier",
    F.when(F.col("outlier_type") < 0.5, 
           F.lit(2.0) + F.rand(seed=456) * F.lit(1.0))  # 2.0 to 3.0
    .otherwise(
           F.lit(0.3) + F.rand(seed=789) * F.lit(0.2))  # 0.3 to 0.5
)

# Calculate new seat values
df_selected = df_selected.withColumn(
    "new_flight_leg_total_seats",
    (F.col("flight_leg_total_seats") * F.col("multiplier")).cast("int")
)

# Create a temporary view for the update
df_updates = df_selected.select(
    "trip_origin_city",
    "trip_destination_city", 
    "flight_leg_departure_date",
    "flight_leg_origin_city",
    "flight_leg_destination_city",
    "new_flight_leg_total_seats",
    "flight_leg_total_seats",
    "multiplier"
)

df_updates.createOrReplaceTempView("anomaly_updates")

print(f"Total rows for 2026-01-29: {df_target_date.count()}")
print(f"Rows selected for anomaly injection (10%): {df_updates.count()}")
print(f"\nSample of updates (showing original vs new seat counts):")
display(df_updates.limit(20))

In [0]:
# Save the updates to a temporary table
df_updates.write.mode("overwrite").saveAsTable("mc.amadeus2.anomaly_updates")



In [0]:
# Use SQL UPDATE with a correlated subquery to update matching rows
# We'll update based on all the key columns plus use LIMIT 1 to handle duplicates
spark.sql("""
MERGE INTO mc.amadeus2.data_jan26 AS target
USING mc.amadeus2.temp_anomaly_updates AS updates
ON target.trip_origin_city = updates.trip_origin_city 
   AND target.trip_destination_city = updates.trip_destination_city 
   AND target.flight_leg_departure_date = updates.flight_leg_departure_date 
   AND target.flight_leg_origin_city = updates.flight_leg_origin_city 
   AND target.flight_leg_destination_city = updates.flight_leg_destination_city
   AND target.flight_leg_total_seats = updates.flight_leg_total_seats
WHEN MATCHED THEN UPDATE SET
  target.flight_leg_total_seats = updates.new_flight_leg_total_seats
""")

# Clean up temp table
#spark.sql("DROP TABLE IF EXISTS mc.amadeus2.temp_anomaly_updates")

print("✓ Anomalies successfully injected into mc.amadeus2.data_jan26")